In [268]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

pd.set_option('display.max_rows', 100)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [269]:
file_path = '/home/carolus/Documents/school/green_ia/data/01_data/01_openfoodfacts_03.jsonl'
num_samples = 500

In [270]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df

In [271]:
df = sample_jsonl_file(file_path, num_samples)

In [272]:
df.tail(60)

,groups,ingredients_temp,packaging,name,ecoscore_tags,categories_temp,ecoscore_score,labels_temp,countries
440,Beverages,"[en:cucumber-juice, en:vegetable, en:fruit-veg...",None,You'Ve Got Kale Juice,[unknown],"[en:plant-based-foods-and-beverages, en:bevera...",NaN,None,united states
441,Sugary snacks,"[fr:f, fr:mini-rochers-praline-noisette-enrobe...",,2 mini rochers,[unknown],"[en:snacks, en:sweet-snacks, en:cocoa-and-its-...",NaN,"[en:organic, en:eu-organic]",france
442,unknown,None,None,Bizcocho redondo manzana,[unknown],None,NaN,None,spain
443,unknown,[en:stee-berggold-chocolatiers-gelee-fruchte-m...,None,Gelee Früchte Mix,[unknown],None,NaN,"[en:vegetarian, en:vegan]",france
444,Fish Meat Eggs,None,Plastic round box,salmon smoked lox sliced,[unknown],"[en:seafood, en:fishes-and-their-products, en:...",NaN,None,united states
445,Fish Meat Eggs,None,,Jambon braisé à griller en tranches,[unknown],"[en:meats-and-their-products, en:meats, en:pre...",NaN,[],france
446,unknown,None,None,Glace Toblerone,[unknown],None,NaN,None,switzerland
447,unknown,None,None,Rote Ruben salat,[unknown],None,NaN,None,germany
448,Fat and sauces,"[en:sunflower-oil, en:oil-and-fat, en:vegetabl...",None,Mayonnaise à la moutarde de Dijon,[d],"[en:condiments, en:sauces, en:mayonnaises, en:...",37.0,"[en:eu-agriculture, en:nutriscore, en:nutrisco...",world
449,unknown,None,None,Pur sirop d’agave,[d],"[en:sweeteners, en:syrups, en:simple-syrups, e...",38.0,"[en:organic, en:eu-organic, en:green-dot, fr:a...",france


In [273]:
uniques_groups = df['groups'].unique()
uniques_groups_list = list(uniques_groups)
for value in uniques_groups_list:
    print(value)

Fruits and vegetables
unknown
Composite foods
Sugary snacks
Milk and dairy products
Cereals and potatoes
Beverages
Salty snacks
Fat and sauces
Alcoholic beverages
Fish Meat Eggs
None


In [274]:
uniques_name = df['name'].unique()
uniques_name_list = list(uniques_name)
for value in uniques_name_list:
    print(value)

Kidsappel gala pommes bio delhaize
Macadamias dry roasted
Royalwrap
Condensed chicken noodle soup
None
Halloumi cheese
Refried Ranchero Beans
Blackberries
Jungle
Funghi champignon
Bijou Cacao
Bifidus Limone
Cracked black pepper
Bark Bites, White & Dark Chocolate Peppermint
Galleta artesana
Latte
Cool fruits Pomme Acérola
Garlic & Herb Chicken Kievs
Riso Cinnamon
Mozzarella fior di latte
Spiral tricolore
Nest Box - Parrot L Shaped Large
Eau minérale
Noci di Sorrento
Finísimo jamón serrano reducido en sal
Miss O ! Saveur Grenade
Oeuf gourmand 285g noir
Sirop saveur Fraise bonbon
Spring wildflower honey
Gateau
R1 Protein
Roscón con nata al Pedro Ximénez
Kuchenglasur Haselnuss
Gaufres molles avec fourrage au foit noisettes
Organic red beet powder
Hot buffalo wings sauce
Calzone Chorizo Picante
Sage et sauvage
Whwy protein 360
Peanut butter on cheese sandwich crackers
Rosé Spätburgunder Trocken 2023
Multivitamin Gummies
Lauretana
Gouda cheese
Les filets
Fiambre Perna  Extra Forno Lenha
Llib

In [277]:
uniques_countries = df['countries'].unique()
uniques_countries_list = list(uniques_countries)
for value in uniques_countries_list:
    print(value)

belgium
united states
france
None
qatar
us
luxembourg
italy
world
spain
united kingdom
germany
brazil
canada
australia
switzerland
morocco
thailand
lithuania
puerto rico
bulgaria
greece
russia
austria
new zealand
turkey
ca
japan
netherlands
ireland
romania
denmark
india
singapore
finland
argentina
cuba
sweden
lebanon


In [275]:
uniques_packaging = df['packaging'].unique()
uniques_packaging_list = list(uniques_packaging)
for value in uniques_packaging_list:
    print(value)

None
Plastique, Sachet, Boîte, Carton
Plastique
Plastique, Carton

Plastique, Bouteille
bottiglia pet
Plastique, Barquette
Verre
Barquette
Plástico, Atmósfera protectora, Refrigerado, Pasteurizado, V gama
Sachet
vin
packed
Métal, Conserve
en:mixed plastic film-packet
Plastic
Plastic, Tray
Bouteille en plastique
Boîte, Conserve, Frais
Plastic, Jug
en:Metal, en:Recyclable Metals, 41 ALU, Recipiente
Bocal en verre, Capsule en métal
Bocal
Plastique, Frais
Verre, Bouteille
Brique, Carton
Стеклянная банка
en:card-box
Cardboard,Cartridge,FSC cardboard
Tetra Pak
Papier, Sachet, Carton
fr:Conteneur, fr:Boîte en carton
Kunststoff, Pappe, 21 PAP, Karton
Plastique, Sachet
Mehrweg Glas
Kunststoff
Stück,Dose
Metall, Wiederverwertbare Metalle, Aluminium, Pappe, Karton
Bouteille, Bouchon de bouteille, Bouchon de bouteille de vin, Bouteille en verre verte, MDP
Papier, Frais
Kunststoff, Pappe, Aluholie
Pappe
Carton
Pappe, Kunststofffolie, Pappschale
Plastic round box
Produkt
Can
Barquette en plastique, 

In [276]:
uniques_ecoscore_tags = df['ecoscore_tags'].unique()
uniques_ecoscore_tags_list = list(uniques_ecoscore_tags)
for value in uniques_ecoscore_tags_list:
    print(value)

TypeError: unhashable type: 'list'

In [ ]:
uniques_ecoscore_score = df['ecoscore_score'].unique()
uniques_ecoscore_score_list = list(uniques_ecoscore_score)
for value in uniques_ecoscore_score_list:
    print(value)

nan
37.0
85.0
23.0
5.0
76.0
27.0
71.0
78.0
15.0
92.0
68.0
34.0
50.0
22.0
99.0
21.0
25.0
36.0
8.0
39.0
84.0
38.0
79.0
45.0
9.0
67.0
72.0
77.0
48.0
13.0
60.0
0.0
64.0
32.0
49.0
65.0
70.0
69.0
18.0
54.0
40.0
44.0
28.0
30.0
73.0
46.0
57.0
59.0
17.0
20.0
16.0
53.0
52.0
61.0


In [ ]:
uniques_ingredients = df['ingredients'].unique()
uniques_ingredients_list = list(uniques_ingredients)
for value in uniques_ingredients_list:
    print(value)

None
honey, added-sugar
peanut, nut, wheat-flour, cereal, flour, wheat, cereal-flour, rice-flour, rice, potato-starch, starch, icing-sugar, added-sugar, disaccharide, sugar, corn-starch, salt, colour, vegetable-oil, oil-and-fat, vegetable-oil-and-fat, mixed-spices, condiment, spice, preservative, onion, vegetable, root-vegetable, onion-family-vegetable, maltodextrin, yeast-extract, yeast, rapeseed-oil, e141ii, e141, palm, palm-oil-and-fat, palm-oil, flavouring, soy-sauce, sauce, soya-bean, legume, pulse, soya, e262ii, e262
glucose-syrup, added-sugar, monosaccharide, glucose, olive-oil, oil-and-fat, vegetable-oil-and-fat, vegetable-oil, sea-salt, salt, concentrated-agave-syrup, disaccharide, agave-syrup, tomato, vegetable, fruit-vegetable, starch, spice, condiment, herb, concentrated-lemon-juice, fruit, juice, fruit-juice, lemon-juice, garlic, root-vegetable, onion-family-vegetable, onion, spice-or-bell-pepper, bell-pepper, pepper, seed, taproot-vegetable, parsnip, bay-leaf, nutmeg-nut,

In [ ]:
uniques_categories = df['categories'].unique()
uniques_categories_list = list(uniques_categories)
for value in uniques_categories_list:
    print(value)

None
beverages, alcoholic-beverages, beers
breakfasts, spreads, sweet-spreads, bee-products, farming-products, sweeteners, honeys
plant-based-foods-and-beverages, plant-based-foods, legumes-and-their-products, snacks, legumes, salty-snacks, nuts-and-their-products, nuts, peanuts
plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetables-based-foods, fruits-based-foods, nuts-and-their-products, dried-products, nuts, dried-plant-based-foods, dried-fruits, dried-mixed-fruits
fish-and-meat-and-eggs, foies-gras, foies-gras-from-ducks, block-of-foie-gras, duck-liver-foie-gras-without-pieces
meats-and-their-products, meats, poultries
plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetables-based-foods, desserts, fruits-based-foods, compotes, apple-compotes, peach-applesauce
plant-based-foods-and-beverages, plant-based-foods, cereals-and-potatoes, meats-and-their-products, cereals-and-their-products, meats, prepared-meats, pastas
meats-and-their-products, meats,

In [ ]:
uniques_labels_note = df['labels_note'].unique()
uniques_labels_note_list = list(uniques_labels_note)
for value in uniques_labels_note_list:
    print(value)

0.0
0.1111111111
1.0
0.4444444444
0.2222222222
0.3333333333
0.8888888889
0.7777777778
0.6666666667
0.5555555556


In [ ]:
df.describe()

,groups,ecoscore_tags,ecoscore_score,labels_note
count,165.000000,110.000000,108.000000,500.000000
mean,0.370438,0.529545,47.305556,0.061111
std,0.286346,0.286812,22.811892,0.134919
min,0.000000,0.000000,0.000000,0.000000
25%,0.111111,0.250000,29.500000,0.000000
50%,0.444444,0.500000,45.500000,0.000000
75%,0.555556,0.750000,68.000000,0.111111
max,1.000000,1.000000,99.000000,1.000000


In [ ]:
def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

def create_treemap_for_column(df, column_name):
    if column_name not in df.columns:
        print(f"warning, column: '{column_name}'does not exists")
        return
    data = df[column_name].value_counts().reset_index()
    data.columns = [column_name, 'count']
    fig = go.Figure(go.Treemap(
        labels=data[column_name],
        parents=[""] * len(data),
        values=data['count'],
        textinfo='label+value',  
    ))
    fig.update_layout(
        title=f'treemap column: {column_name}',
        paper_bgcolor='black',  
        plot_bgcolor='grey',   
        font_color='white'      
    )
    fig.show()

In [ ]:
colonnes_a_travailler = ['ecoscore_tags', 'ecoscore_score', 'countries', 'labels_note']  
df = load_jsonl_to_dataframe(file_path)
for column in colonnes_a_travailler:
    create_treemap_for_column(df, column)

In [ ]:
def count_unique_values(jsonl_file_path, colonnes_a_travailler):
    data = []
    with open(jsonl_file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    unique_counts = {}
    for column in colonnes_a_travailler:
        if column in df.columns:
            unique_count = df[column].nunique()
            if pd.api.types.is_numeric_dtype(df[column]):
                min_val = df[column].min()
                max_val = df[column].max()
                unique_counts[column] = {
                    'unique_count': unique_count,
                    'min': min_val,
                    'max': max_val
                }
            else:
                unique_counts[column] = {
                    'unique_count': unique_count
                }
        else:
            unique_counts[column] = 'Column not found in data'
    return unique_counts

In [ ]:
unique_values = count_unique_values(file_path, colonnes_a_travailler)
print(unique_values)

{'ecoscore_tags': {'unique_count': 6, 'min': 0.0, 'max': 1.0}, 'ecoscore_score': {'unique_count': 102, 'min': 0.0, 'max': 100.0}, 'countries': {'unique_count': 11710}, 'labels_note': {'unique_count': 10, 'min': 0.0, 'max': 1.0}}
